# YOLO v3 Finetuning on AWS

This series of notebooks demonstrates how to finetune pretrained YOLO v3 (aka YOLO3) using MXNet on AWS.

**This notebook** walks through using the [SageMaker Ground Truth](https://aws.amazon.com/sagemaker/groundtruth/) tool to annotate training and validation data sets.

**Follow-on** the content of the notebooks shows:

* How to use MXNet YOLO3 pretrained model
* How to use Deep SORT with MXNet YOLO3
* How to create Ground-Truth dataset from images the model mis-detected
* How to finetune the model using the created dataset
* Load your finetuned model and Deploy Sagemaker-Endpoint with it using CPU instance.
* Load your finetuned model and Deploy Sagemaker-Endpoint with it using GPU instance.

## Pre-requisites

This notebook is designed to be run in Amazon SageMaker. To run it (and understand what's going on), you'll need:

* Basic familiarity with Python, [MXNet](https://mxnet.apache.org/), [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon Sagemaker](https://aws.amazon.com/sagemaker/)
* To create an **S3 bucket** in the same region, and ensure the SageMaker notebook's role has access to this bucket.
* Sufficient [SageMaker quota limits](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html#limits_sagemaker) set on your account to run GPU-accelerated spot training jobs.

## Cost and runtime

Depending on your configuration, this demo may consume resources outside of the free tier but should not generally be expensive because we'll be training on a small number of images. You might wish to review the following for your region:

* [Amazon SageMaker pricing](https://aws.amazon.com/sagemaker/pricing/)
* [SageMaker Ground Truth pricing](https://aws.amazon.com/sagemaker/groundtruth/pricing/)

The standard `ml.t2.medium` instance should be sufficient to run the notebooks.

We will use GPU-accelerated instance types for training and hyperparameter optimization, and use spot instances where appropriate to optimize these costs.

As noted in the step-by-step guidance, you should take particular care to delete any created SageMaker real-time prediction endpoints when finishing the demo.

# Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [7]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import re
import os
import json
from glob import glob
from pprint import pprint
from matplotlib import pyplot as plt

# External Dependencies:
import boto3
import imageio
import sagemaker
from botocore.exceptions import ClientError

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
BUCKET_NAME = sagemaker.Session().default_bucket()
%store BUCKET_NAME

REGION = sagemaker.Session().boto_region_name
%store REGION

IMAGE_PREFIX = 'images'
%store IMAGE_PREFIX

MODELS_PREFIX = 'models'
%store MODELS_PREFIX

CLASS_NAMES = ['person']
%store CLASS_NAMES

BATCH_NAME = 'yolo-workshop-batch'
%store BATCH_NAME

Stored 'BUCKET_NAME' (str)
Stored 'REGION' (str)
Stored 'IMAGE_PREFIX' (str)
Stored 'MODELS_PREFIX' (str)
Stored 'CLASS_NAMES' (list)
Stored 'BATCH_NAME' (str)


In [9]:
session = boto3.session.Session()
region = session.region_name
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)
smclient = session.client('sagemaker')

In [10]:
print(bucket.name)

sagemaker-ap-northeast-2-929831892372


## Step 1: Create bucket

In this notebook, we are going to label the mis-detected images using Sagemaker Ground Truth.

Most of the Sagemaker services are needed the data on the S3 to use.


In [11]:
try:
    bucket.create(
        ACL='private',
        CreateBucketConfiguration={
            'LocationConstraint': REGION,
        },
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print('Bucket have been already created..')
    else:
        raise e

Bucket have been already created..


In [12]:
# set cors to bucket

cors_config = {
    'CORSRules': [{
        'AllowedHeaders': ['Authorization'],
        'AllowedMethods': ['GET', 'PUT'],
        'AllowedOrigins': ['*'],
        'ExposeHeaders': ['GET', 'PUT'],
    }]
}
cors = bucket.Cors()
cors.put(CORSConfiguration=cors_config)

{'ResponseMetadata': {'RequestId': 'CD6C26699D976BE5',
  'HostId': 'Kvw4olpwumeNkR7xaEks8FlK4/We4QKPTDpGJuOzgFKts2pegWQBsxgy8SY1UWJfwwe3UN4jwtY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Kvw4olpwumeNkR7xaEks8FlK4/We4QKPTDpGJuOzgFKts2pegWQBsxgy8SY1UWJfwwe3UN4jwtY=',
   'x-amz-request-id': 'CD6C26699D976BE5',
   'date': 'Mon, 16 Mar 2020 07:20:23 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

## Step 2: Upload images to S3

Let's say the mis-detected images are stored at `/Users/dongkyl/Documents/git/mxnet-deepsort-yolo3/images`, below code will upload your images onto S3.

In [13]:
local_image_path = '/Users/dongkyl/Documents/git/mxnet-deepsort-yolo3/images'
print(local_image_path)

/Users/dongkyl/Documents/git/mxnet-deepsort-yolo3/images


on S3, the path for images would be `/{BATCH_NAME}/{IMAGE_PREFIX}`. 

By this, the `full s3 path` of the image will be `s3://{BUCKET_NAME}/{BATCH_NAME}/{IMAGE_PREFIX}/{FILENAME}`.

In [14]:
upload_path = f'{BATCH_NAME}/{IMAGE_PREFIX}'
print(upload_path)

yolo-workshop-batch/images


Now, we are going to upload images to bucket to make `input.manifest` that is for Ground-Truth labeling job. And, of course, the images will also be used in finetuning too.

In [15]:
filenames = []

for file_path in glob(f'{local_image_path}/missed-*.jpg'):
    # we are going to label on entire scene, not detected area
    file_path = re.sub(r'missed-(\d+)-\d+', 'orig-\\1', file_path)
    
    filename = file_path.replace('orig-', '').rsplit('/', 1)[-1]
    bucket.upload_file(file_path, f'{upload_path}/{filename}')
    filenames.append(filename)
    print(f'uploaded {filename}...')

uploaded 235.jpg...
uploaded 88.jpg...
uploaded 256.jpg...
uploaded 349.jpg...
uploaded 136.jpg...
uploaded 112.jpg...
uploaded 328.jpg...
uploaded 274.jpg...
uploaded 217.jpg...
uploaded 352.jpg...
uploaded 250.jpg...
uploaded 292.jpg...
uploaded 148.jpg...
uploaded 130.jpg...
uploaded 268.jpg...
uploaded 196.jpg...
uploaded 76.jpg...
uploaded 316.jpg...
uploaded 154.jpg...
uploaded 94.jpg...
uploaded 229.jpg...
uploaded 310.jpg...
uploaded 244.jpg...
uploaded 346.jpg...
uploaded 286.jpg...
uploaded 124.jpg...
uploaded 262.jpg...
uploaded 100.jpg...
uploaded 340.jpg...
uploaded 280.jpg...
uploaded 106.jpg...
uploaded 223.jpg...
uploaded 160.jpg...
uploaded 322.jpg...
uploaded 118.jpg...
uploaded 304.jpg...
uploaded 82.jpg...
uploaded 202.jpg...
uploaded 298.jpg...
uploaded 142.jpg...


## Step 3: Generate input.manifest

In order to set up the Sagemaker Ground Truth labeling job, you should make a manifest file that contains the list of the files on S3.

The [**manifest**](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-input.html) file is just list of dictionaries and each row must contain one of the key `source-ref`. `source-ref` value is going to be the `full s3 path` of the image file, that is mentioned above.

We are going to generate input manifest and place it into `{BATCH_NAME}/manifests/input.manifest`. And then upload it onto S3 bucket with same path. 

The full path for manifest would be `s3://{BUCKET_NAME}/{BATCH_NAME}/manifests/input.manifest`.

In [16]:
os.makedirs(f'{BATCH_NAME}/manifests', exist_ok=True)
input_manifest_loc = f'{BATCH_NAME}/manifests/input.manifest'

with open(input_manifest_loc, 'w') as fp:
    for filename in filenames:
        source_ref = f's3://{bucket.name}/{upload_path}/{filename}'
        fp.write(json.dumps({'source-ref': source_ref})+'\n')

bucket.upload_file(input_manifest_loc, input_manifest_loc)

You can visit the [**AWS S3 Console**](https://s3.console.aws.amazon.com/) to make sure images are uploaded successfully.

Of course, you can display the list of files on the notebook using boto3.

In [17]:
for obj in bucket.objects.filter(Prefix=f'{BATCH_NAME}'):
    print(obj.key)

yolo-workshop-batch/images/100-2.jpg
yolo-workshop-batch/images/100.jpg
yolo-workshop-batch/images/106-2.jpg
yolo-workshop-batch/images/106.jpg
yolo-workshop-batch/images/112-2.jpg
yolo-workshop-batch/images/112.jpg
yolo-workshop-batch/images/118-2.jpg
yolo-workshop-batch/images/118.jpg
yolo-workshop-batch/images/124-2.jpg
yolo-workshop-batch/images/124.jpg
yolo-workshop-batch/images/130-2.jpg
yolo-workshop-batch/images/130.jpg
yolo-workshop-batch/images/136-2.jpg
yolo-workshop-batch/images/136.jpg
yolo-workshop-batch/images/142-2.jpg
yolo-workshop-batch/images/142.jpg
yolo-workshop-batch/images/148-2.jpg
yolo-workshop-batch/images/148.jpg
yolo-workshop-batch/images/154-2.jpg
yolo-workshop-batch/images/154.jpg
yolo-workshop-batch/images/160-2.jpg
yolo-workshop-batch/images/160.jpg
yolo-workshop-batch/images/196-3.jpg
yolo-workshop-batch/images/196.jpg
yolo-workshop-batch/images/202-3.jpg
yolo-workshop-batch/images/202.jpg
yolo-workshop-batch/images/217-3.jpg
yolo-workshop-batch/images/

## Step 4: Setup Sagemker Ground Trutch Labeling workforce

Sagemaker Ground Truth workforce gives you 3 options such as,

- Amazon Mechanical Turk
- Private
- Vendor

for more details, visit [**Use Amazon SageMaker Ground Truth for Labeling**](https://docs.aws.amazon.com/sagemaker/latest/dg/sms.html).

In this notebook, We are going to use *Private Workforce* to label bounding boxes to our data. With Private workforce you can make your employees or contractors handling data within your organization.

**in the [AWS console](https://console.aws.amazon.com)**,
Under *Services* go to *Amazon SageMaker*, and select *Ground Truth > Labeling workforces* from the side-bar menu on the left. And select the *Private* tab on the top, click the button *Create private team* to create private workforce.

<img src="Assets/PrivateWorkforce.png" />

By Creating workforce,

- Private tab menu displays link under *Labeling portal sign-in URL*
- Workers will got a email with temporary password

When worker visit the link and login for the first time(with email as username and password as temporary password), worker has to reset the password.

After resetting password, worker is finally ready to work!

## Step 5: Set up the SageMaker Ground Truth labeling job

Now that our images and a manifest file listing them are ready in S3, we'll set up the Ground Truth labeling job **in the [AWS console](https://console.aws.amazon.com)**.

Under *Services* go to *Amazon SageMaker*, and select *Ground Truth > Labeling Jobs* from the side-bar menu on the left.

### Job Details

Click the **Create labeling job** button, and you'll be asked to specify job details as follows:

* **Job name:** Choose a name to identify this labeling job, e.g. `yolo-workshop-batch-0`
* **Label name (The override checkbox):** Consider overriding this to `labels`
* **Input data location:** The path to the input manifest file in S3 (see output above)
* **Output data location:** The path to store labeled dataset in S3 (e.g.  *s3://{BUCKET_NAME}/{BATCH_NAME}/annotations*)
* **IAM role:** If you're not sure whether your existing roles have the sufficient permissions for Ground Truth, select the options to create a new role
* **Task type:** Image > Bounding box

<img src="Assets/SetupGroundTruth.png"/>

All other settings can be left as default. Record your choices for the label name and output data location below, because we'll need these later:

In [18]:
job_name = 'yolo-job-0'
%store job_name

Stored 'job_name' (str)


In [19]:
print(f'intpu_dataset_location: s3://{bucket.name}/{input_manifest_loc}')
print(f'output_dataset_location: s3://{bucket.name}/annotations')

intpu_dataset_location: s3://sagemaker-ap-northeast-2-929831892372/yolo-workshop-batch/manifests/input.manifest
output_dataset_location: s3://sagemaker-ap-northeast-2-929831892372/annotations


### Workers
On the next screen, we'll configure who will annotate our data: Ground Truth allows you to define your own in-house Private Workforces; use Vendor Managed Workforces for specialist tasks; or use the public workforce provided by Amazon Mechanical Turk.

Select Private worker type, and you'll be prompted either to select from your existing private workforces, or create a new one if none exist.

To create a new private workforce if you need, simply follow the UI workflow with default settings. It doesn't matter what you call the workforce, and you can create a new Cognito User Group to define the workforce. Add yourself to the user pool by adding your email address: You should receive a confirmation email shortly with a temporary password and a link to access the annotation portal.

Automatic data labeling is applicable only for data sets over 1000 samples, so leave this turned off for now.

<img src="Assets/Workers.png" />

### Labeling Tool
Since you'll be labelling the data yourself, a brief description of the task should be fine in this case. When using real workforces, it's important to be really clear in this section about the task requirements and best practices - to ensure consistency of annotations between human workers.

For example: In the common case where we see a pair of boots from the side and one is almost entirely obscured, how should the image be annotated? Should model cats count, or only real ones?

The most important configuration here is to set the options to be the same as our CLASS_NAMES, we have only one label in this workshop *Person*.

<img src="Assets/LabelingTool.png" />

Take some time to explore the other options for configuring the annotation tool; and when you're ready click "Create" to launch the labeling job.

## Step 5: Label those images!

Follow the link you received in your workforce invitation email to the workforce's **labeling portal**, and log in with the default password given in the email (which you'll be asked to change).

If you lose the portal link, you can always retrieve it through the *Ground Truth > Labeling Workforces* menu in the SageMaker console: Near the top of the summary of private workforces.

New jobs can sometimes take a minute or two to appear for workers. Select the job and click "Start working" to enter the labeling tool.

<img src="Assets/WorkerLabelingJobs.png"/>

once the labeling job is started, you will see this labeling job web page..

<img src="Assets/WorkerLabelingPage.png"/>

Note that you can check on the progress of labelling jobs through the APIs as well as in the AWS console.
After few seconds from workers done their labeling job, the status will be changed to *Completed*

In [55]:
smclient.describe_labeling_job(LabelingJobName=job_name)['LabelingJobStatus']

'Completed'

## Step 6: Check the labeling results

when your workers done their job, *output.manifest* will be generated into following path.

*s3://{BUCKET_NAME}/annotations/{job_name}/manifests/output/output.manifest*

In [20]:
output_manifest_path = f'annotations/{job_name}/manifests/output/output.manifest'
output_manifest_obj = bucket.Object(output_manifest_path)
for el in map(json.loads, output_manifest_obj.get()['Body'].read().decode('utf-8').split('\n')):
    pprint(el)
    break

{'labels': {'annotations': [{'class_id': 0,
                             'height': 230,
                             'left': 90,
                             'top': 45,
                             'width': 182},
                            {'class_id': 0,
                             'height': 231,
                             'left': 150,
                             'top': 0,
                             'width': 174}],
            'image_size': [{'depth': 3, 'height': 320, 'width': 427}]},
 'labels-metadata': {'class-map': {'0': 'Person'},
                     'creation-date': '2020-03-14T12:51:04.692288',
                     'human-annotated': 'yes',
                     'job-name': 'labeling-job/yolo-job-0',
                     'objects': [{'confidence': 0.09}, {'confidence': 0.09}],
                     'type': 'groundtruth/object-detection'},
 'source-ref': 's3://sagemaker-ap-northeast-2-929831892372/yolo-workshop-batch/images/235.jpg'}


As you can see, top level keys are 2, *source-ref* and *labels* respectively. the key *label* is the name you gave when setting up the labeling job. which is containing information of all bound-boxes about *source-ref* image.

In next chapter, we will create Sagemaker Hyperparmeter Optimization(a.k.a HPO) Job with this *output.manifest*.